## Working with ICESat-2 Data's Nested Variables
### Get a list of available variables and choose the ones you want to work with

This notebook illustrates the use of icepyx for managing lists of available and wanted ICESat-2 data variables.
The two use cases for variable management within your workflow are:
1. During the data order and download (e.g. via NSIDC DAAC) and data access (e.g. via the cloud) process.
2. When reading in data to a Python object (whether from local files or the cloud).

A given ICESat-2 product may have over 200 variable + path combinations.
icepyx includes a custom `Variables` module that is "aware" of the ATLAS sensor and how the ICESat-2 data products are stored.
The module can be accessed independently, but is optimally used as a component of a `Query` object (Case 1) or `Reader` object (Case 2).

This notebook illustrates in detail how the `Variables` module behaves using a `Query` data access example.
However, module usage is analogous through an icepyx ICESat-2 `Reader` object.
More detailed example workflows specifically for the [query](https://github.com/icesat2py/icepyx/blob/main/doc/examples/ICESat-2_DAAC_DataAccess_Example.ipynb) and [reader](https://github.com/icesat2py/icepyx/blob/main/doc/examples/ICESat-2_Data_Read-in_Example.ipynb) components of icepyx are available as separate Jupyter Notebooks.

Questions? Be sure to check out the FAQs throughout this notebook, indicated as italic headings.

#### Credits
* notebook by: Jessica Scheick

### _Why do ICESat-2 products need a custom variable manager?_

ICESat-2 data is natively stored in a nested file format called hdf5.
Much like a directory-file system on a computer, each variable (file) has a unique path through the heirarchy (directories) within the file.
Thus, some variables (e.g. `'latitude'`, `'longitude'`) have multiple paths (one for each of the six beams in most products).

It can be confusing and cumbersome to comb through the 200+ variable and path combinations.
The icepyx `Variables` module makes it easier for users to quickly find and extract the specific variables they would like to work with across multiple beams, keywords, and variables and provides reader-friendly formatting to browse variables.
A future development goal for `icepyx` includes developing an interactive widget to further improve the user experience.
For data read-in, additional tools are available to target specific beam characteristics (e.g. strong versus weak beams).

To increase readability, some display options (2 and 3, below) show the 200+ variable + path combinations as a dictionary where the keys are variable names and the values are the paths to that variable.



# Start editing content/text here!


### Import packages, including icepyx

In [8]:
import icepyx as ipx

import numpy as np
import xarray as xr
import pandas as pd

import h5py
import os,json
from pprint import pprint

### Create a query object and log in to Earthdata

For this example, we'll be working with a sea ice product (ATL09) for an area along West Greenland (Disko Bay).

In [24]:
region_a = ipx.Query('ATL09',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59')

In [6]:
region_a.earthdata_login('username','jbscheick@gmail.com')

Earthdata Login password: ········
['Invalid username or password, please retry.']
Please re-enter your Earthdata user ID: jessica.scheick
Earthdata Login password: ········


## About Data Variables in a query object
There are two possible variable parameters associated with each ```query``` object.
1. `order_vars`, which is for interacting with variables during data querying, ordering, and downloading activities. `order_vars.wanted` holds the user's list to be submitted to the NSIDC subsetter and download a smaller, reproducible subset of the data product.
2. `file_vars`, which is for interacting with variables associated with local files [not yet implemented].

Each variables parameter (which is actually an associated Variables class object) has methods to:
* get available variables, either available from the NSIDC or the file (`avail()` method/attribute).
* append new variables to the wanted list (`append()` method).
* remove variables from the wanted list (`remove()` method).

Each variables instance also has a set of attributes, including `avail` and `wanted` to indicate the list of variables that is available (unmutable, or unchangeable, as it is based on the input product specifications or files) and the list of variables that the user would like extracted (updateable with the `append` and `remove` methods), respectively. We'll showcase the use of all of these methods and attributes below.

### ICESat-2 data variables
ICESat-2 data is natively stored in a nested file format called hdf5. Much like a directory-file system on a computer, each variable (file) has a unique path through the heirarchy (directories) within the file. Thus, some variables (e.g. `'latitude'`, `'longitude'`) have multiple paths (one for each of the six beams in most products). 

To increase readability, some display options (2 and 3, below) show the 200+ variable + path combinations as a dictionary where the keys are variable names and the values are the paths to that variable.

### Determine what variables are available
There are multiple ways to get a complete list of available variables.

1. `region_a.order_vars.avail`, a list of all valid path+variable strings
2. `region_a.show_custom_options(dictview=True)`, all available subsetting options
3. `region_a.order_vars.parse_var_list(region_a.order_vars.avail)`, a dictionary of variable:paths key:value pairs

In [ ]:
region_a.order_vars.avail()

By passing the boolean `options=True` to the `avail` method, you can obtain lists of unique possible variable inputs (var_list inputs) and path subdirectory inputs (keyword_list and beam_list inputs) for your data product. These can be helpful for building your wanted variable list.

In [ ]:
region_a.order_vars.avail(options=True)

### _Why not just download all the data and subset locally? What if I need more variables/granules?_

Taking advantage of the NSIDC subsetter is a great way to reduce your download size and thus your download time and the amount of storage required, especially if you're storing your data locally during analysis. By downloading your data using `icepyx`, it is easy to go back and get additional data with the same, similar, or different parameters (e.g. you can keep the same spatial and temporal bounds but change the variable list). Related tools (e.g. [`captoolkit`](https://github.com/fspaolo/captoolkit)) will let you easily merge files if you're uncomfortable merging them during read-in for processing.

### Building your wanted variable list

Now that you know which variables and path components are available for your product, you need to build a list of the ones you'd like included. There are several options for generating your initial list as well as modifying it, giving the user complete control over the list submitted.

The options for building your initial list are:
1. Use a default list for the product (not yet fully implemented across all products. Have a default variable list for your field/product? Submit a pull request or post it as an issue on GitHub!)
2. Provide a list of variable names
3. Provide a list of profiles/beams or other path keywords, where "keywords" are simply the unique subdirectory names contained in the full variable paths of the product. A full list of available keywords for the product is displayed in the error message upon entering `keyword_list=['']` into the `append` function (see below for an example) or by running `region_a.order_vars.avail(options=True)`, as above

Note: all products have a short list of "mandatory" variables/paths (containing spacecraft orientation and time information needed to convert the data's `delta_time` to a readable datetime) that are automatically added to any built list. If you have any recommendations for other variables that should always be included (e.g. uncertainty information), please let us know!

Examples of using each method to build and modify your wanted variable list are below.

In [ ]:
region_a.order_vars.wanted

In [ ]:
region_a.order_vars.append(defaults=True)
pprint(region_a.order_vars.wanted)

The keywords available for this product are shown in the error message upon entering a blank keyword_list, as seen in the next cell.

In [ ]:
region_a.order_vars.append(keyword_list=[''])

### Modifying your wanted variable list

Generating and modifying your variable request list, which is stored in `region_a.order_vars.wanted`, is controlled by the `append` and `remove` functions that operate on `region_a.order_vars.wanted`. The input options to `append` are as follows (the full documentation for this function can be found by executing `help(region_a.order_vars.append)`).
* `defaults` (default False) - include the default variable list for your product (not yet fully implemented for all products; please submit your default variable list for inclusion!)
* `var_list` (default None) - list of variables (entered as strings)
* `beam_list` (default None) - list of beams/profiles (entered as strings)
* `keyword_list` (default None) - list of keywords (entered as strings); use `keyword_list=['']` to obtain a list of available keywords

Similarly, the options for `remove` are:
* `all` (default False) - reset `region_a.order_vars.wanted` to None
* `var_list` (as above)
* `beam_list` (as above)
* `keyword_list` (as above)

In [ ]:
region_a.order_vars.remove(all=True)
pprint(region_a.order_vars.wanted)

### Examples
Below are a series of examples to show how you can use `append` and `remove` to modify your wanted variable list. For clarity, `region_a.order_vars.wanted` is cleared at the start of many examples. However, multiple `append` and `remove` commands can be called in succession to build your wanted variable list (see Examples 3+)

#### Example 1: choose variables
Add all `latitude` and `longitude` variables

In [ ]:
region_a.order_vars.append(var_list=['latitude','longitude'])
pprint(region_a.order_vars.wanted)

#### Example 2: specify beams/profiles and variable
Add `latitude` for only `profile_1` and `profile_2`

In [ ]:
region_a.order_vars.remove(all=True)
pprint(region_a.order_vars.wanted)

In [ ]:
var_dict = region_a.order_vars.append(beam_list=['profile_1','profile_2'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

#### Example 3: add/remove selected beams+variables
Add `latitude` for `profile_3` and remove it for `profile_2`

In [ ]:
region_a.order_vars.append(beam_list=['profile_3'],var_list=['latitude'])
region_a.order_vars.remove(beam_list=['profile_2'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

#### Example 4: `keyword_list`
Add `latitude` for all profiles and with keyword `low_rate`

In [ ]:
region_a.order_vars.append(var_list=['latitude'],keyword_list=['low_rate'])
pprint(region_a.order_vars.wanted)

#### Example 5: target a specific variable + path
Remove `'profile_1/high_rate/latitude'` (but keep `'profile_3/high_rate/latitude'`)

In [ ]:
region_a.order_vars.remove(beam_list=['profile_1'], var_list=['latitude'], keyword_list=['high_rate'])
pprint(region_a.order_vars.wanted)

#### Example 6: add variables not specific to beams/profiles
Add `rgt` under `orbit_info`.

In [ ]:
region_a.order_vars.append(keyword_list=['orbit_info'],var_list=['rgt'])
pprint(region_a.order_vars.wanted)

#### Example 7: add all variables+paths of a group
In addition to adding specific variables and paths, we can filter all variables with a specific keyword as well. Here, we add all variables under `orbit_info`. Note that paths already in `region_a.order_vars.wanted`, such as `'orbit_info/rgt'`, are not duplicated.

In [ ]:
region_a.order_vars.append(keyword_list=['orbit_info'])
pprint(region_a.order_vars.wanted)

#### Example 8: add all possible values for variables+paths
Append all `longitude` paths and all variables/paths with keyword `high_rate`.
Simlarly to what is shown in Example 4, if you submit only one `append` call as `region_a.order_vars.append(var_list=['longitude'], keyword_list=['high_rate'])` rather than the two `append` calls shown below, you will only add the variable `longitude` and only paths containing `high_rate`, not ALL paths for `longitude` and ANY variables with `high_rate` in their path.

In [ ]:
region_a.order_vars.append(var_list=['longitude'])
region_a.order_vars.append(keyword_list=['high_rate'])
pprint(region_a.order_vars.wanted)

#### Example 9: remove all variables+paths associated with a beam
Remove all paths for `profile_1` and `profile_3`

In [ ]:
region_a.order_vars.remove(beam_list=['profile_1','profile_3'])
pprint(region_a.order_vars.wanted)

#### Example 10: generate a default list for the rest of the tutorial
Generate a reasonable variable list prior to download

In [ ]:
region_a.order_vars.remove(all=True)
region_a.order_vars.append(defaults=True)
pprint(region_a.order_vars.wanted)

## Applying variable subsetting to your order and download

In order to have your wanted variable list included with your order, you must pass it as a keyword argument to the `subsetparams()` attribute or the `order_granules()` or `download_granules()` (which calls `order_granules` under the hood if you have not already placed your order) functions.

In [ ]:
region_a.subsetparams(Coverage=region_a.order_vars.wanted)

Or, you can put the `Coverage` parameter directly into `order_granules`:
`region_a.order_granules(Coverage=region_a.order_vars.wanted)`

However, then you cannot view your subset parameters (`region_a.subsetparams`) prior to submitting your order.

In [ ]:
region_a.order_granules()# <-- you do not need to include the 'Coverage' kwarg to
                             # order if you have already included it in a call to subsetparams

In [ ]:
region_a.download_granules('/home/jovyan/icepyx/dev-notebooks/vardata') # <-- you do not need to include the 'Coverage' kwarg to
                             # download if you have already submitted it with your order

### _Why does the subsetter say no matching data was found?_
Sometimes, chunks (granules) returned in our initial search end up not containing any data in our specified area of interest. This is because the initial search is completed using summary metadata for a chunk. You've likely encountered this before when viewing available imagery online: your spatial search turns up a bunch of images with only a few border or corner pixels, maybe even in no data regions, in your area of interest. Thus, when you go to extract the data from the area you want (i.e. spatially subset it), you don't get any usable data from that image.

## Check the variable list in your downloaded file

Compare the available variables associated with the full product relative to those in your downloaded data file.

In [ ]:
# put the full filepath to a data file here. You can get this in JupyterHub by navigating to the file,
# right clicking, and selecting copy path. Then you can paste the path in the quotes below.
fn = ''

#### Check the downloaded data
Get all `latitude` variables in your downloaded file:

In [ ]:
varname = 'latitude'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)
    
for tvar in varlist:
    vpath,vn = os.path.split(tvar)
    if vn==varname: print(tvar) 

#### Compare to the variable paths available in the original data

In [ ]:
region_a.order_vars.parse_var_list(region_a.order_vars.avail)[0][varname]